# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

PARALLELISM = 5

import os
os.environ['SNORKELDB'] = 'postgres:///semparse_cdr'

from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print 'Training set:\t{0} candidates'.format(len(train))
print 'Dev set:\t{0} candidates'.format(len(dev))
print 'Test set:\t{0} candidates'.format(len(test))    

Training set:	8272 candidates
Dev set:	888 candidates
Test set:	4620 candidates


In [3]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, split=0)

# (NEW) Featurize

In [6]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

In [ ]:
# %time F_train = featurizer.apply(split=0, parallelism=PARALLELISM)
# F_train

In [ ]:
# %time F_dev  = featurizer.apply_existing(split=1, parallelism=PARALLELISM)
# F_dev

In [ ]:
# %time F_test = featurizer.apply_existing(split=2, parallelism=PARALLELISM)
# F_test

In [7]:
F_train = featurizer.load_matrix(session, split=0)
F_train

<8272x122840 sparse matrix of type '<type 'numpy.float64'>'
	with 448906 stored elements in Compressed Sparse Row format>

In [8]:
F_dev = featurizer.load_matrix(session, split=1)
F_dev

<888x122840 sparse matrix of type '<type 'numpy.float64'>'
	with 27734 stored elements in Compressed Sparse Row format>

In [9]:
F_test = featurizer.load_matrix(session, split=2)
F_test

<4620x122840 sparse matrix of type '<type 'numpy.float64'>'
	with 138487 stored elements in Compressed Sparse Row format>

# Training Logistic Regression

In [10]:
from snorkel.annotations import load_gold_labels
L_gold_train = load_gold_labels(session, annotator_name='gold', split=0)
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [11]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

In [12]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, F_train, train_marginals, [rate_param, l1_param, l2_param], n=20)

Initialized RandomSearch search of size 20. Search space size = 125.


In [13]:
searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=True, print_freq=25)

[1] Testing lr = 1.00e-02, l1_penalty = 1.00e-03, l2_penalty = 1.00e-06
[SparseLR] lr=0.01 l1=0.001 l2=1e-06
[SparseLR] Building model
[SparseLR] Training model
[SparseLR] #examples=5822  #epochs=50  batch size=100
[SparseLR] Epoch 0 (0.55s)	Avg. loss=0.883060	NNZ=122809
[SparseLR] Epoch 25 (12.48s)	Avg. loss=-54.486484	NNZ=122634
[SparseLR] Epoch 49 (24.09s)	Avg. loss=-106.742109	NNZ=122400
[SparseLR] Training done (24.09s)
[SparseLR] Model saved. To load, use name
		SparseLR_0
[2] Testing lr = 1.00e-03, l1_penalty = 1.00e-03, l2_penalty = 1.00e-04
[SparseLR] lr=0.001 l1=0.001 l2=0.0001
[SparseLR] Building model
[SparseLR] Training model
[SparseLR] #examples=5822  #epochs=50  batch size=100
[SparseLR] Epoch 0 (0.55s)	Avg. loss=0.669916	NNZ=122834
[SparseLR] Epoch 25 (12.58s)	Avg. loss=-5.457271	NNZ=122838
[SparseLR] Epoch 49 (24.55s)	Avg. loss=-10.798464	NNZ=122825
[SparseLR] Training done (24.55s)
[3] Testing lr = 1.00e-03, l1_penalty = 1.00e-02, l2_penalty = 1.00e-06
[SparseLR] lr=0

,lr,l1_penalty,l2_penalty,Prec.,Rec.,F1
17,0.000001,0.000010,0.010000,0.341981,0.979730,0.506993
14,0.000100,0.000100,0.000001,0.359375,0.854730,0.506000
4,0.000001,0.001000,0.000010,0.339513,0.989865,0.505608
19,0.000100,0.010000,0.010000,0.355153,0.861486,0.502959
7,0.000010,0.010000,0.000010,0.334842,1.000000,0.501695
16,0.000010,0.000001,0.000100,0.334463,1.000000,0.501270
8,0.000010,0.001000,0.000100,0.334463,1.000000,0.501270
12,0.000100,0.000001,0.000010,0.351573,0.868243,0.500487
9,0.000010,0.001000,0.000100,0.333333,1.000000,0.500000
10,0.000100,0.001000,0.000001,0.351801,0.858108,0.499018


In [14]:
TP, FP, TN, FN = disc_model.score(session, F_dev, L_gold_dev)

Scores (Un-adjusted)
Pos. class accuracy: 0.98
Neg. class accuracy: 0.0574
Precision            0.342
Recall               0.98
F1                   0.507
----------------------------------------
TP: 290 | FP: 558 | TN: 34 | FN: 6



In [15]:
_, _, _, _ = disc_model.score(session, F_test, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.969
Neg. class accuracy: 0.0347
Precision            0.327
Recall               0.969
F1                   0.488
----------------------------------------
TP: 1458 | FP: 3007 | TN: 108 | FN: 47



# Part V: Training an LSTM
In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

In [ ]:
from snorkel.contrib.learning import reLSTM

lstm = reLSTM()
%time lstm.train(
    train, train_marginals, lr=0.005, dim=200, n_epochs=30,
    dropout_rate=0.5, rebalance=0.25, print_freq=5
)

### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did.

In [ ]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

In [ ]:
%time _, _, _, _ = lstm.score(session, test, L_gold_test)